## Cyclic ID Algorithm Implementation with Examples

##### Based on: Forré & Mooij (2019) - "Causal Calculus in the Presence of Cycles, Latent Confounders and Selection Bias"


### 1. Introduction & Setup

#### 1.1 Why do we need an algorithm for Cyclic Graphs?

Traditional causal inference methods assume **acyclic graphs (DAGs)**, which have no feedback loops allowed in them. However, more realistic systems and networks often do include these cycles. Some examples include:

* Gene regulatory networks (GRNs): Gene X activates gene Y, which activates gene Z, which feeds back to activate gene X again. 

* Economic systems: Interest rates affect inflation, which affects interest rates.

* Neural networks: Recurrent connections create feedback loops.

The implementation of this **generalized ID** algorithm for input-output Structural Causal Models (ioSCMs) handles:

* Cycles (feedback loops)

* Latent confounders (unmeasured causes)

* Arbitrary probability distributions


#### 1.2 Key Question

Given a directed mixed graph $G$ with cycles, observational data $P(V)$ and a causal query: "What is $P(Y | \text{do}(X))$?

The main question we ask with this algorithm is ***can we identify the causal effect $P(Y | \text{do}(X))$ from the observational distribution $P(V)$ alone?***

The **generalized ID algorithm** answers this question for cyclic graphs. 


In [6]:
# importing packages and y0 module

from IPython.display import Markdown, display

from y0.algorithm.ioscm.utils import (
    get_apt_order,
    get_consolidated_district,
    get_graph_consolidated_districts,
    get_strongly_connected_components,
    get_vertex_consolidated_district,
    is_apt_order,
)
from y0. dsl import P, Variable
from y0.graph import NxMixedGraph

#### 1.3 Example Graphs to Demonstrate the Need for this algorithm

Here are two examples that illustrate why we need cyclic identification. 

#### Example 1: Simple Feedback Loop

A simple non-identifiable cyclic structure:

![Feedback Loop graph](feedback_loop1.png)


**This is an example of the simplest cyclic structure. Two variables that affect each other simultaneously.**

- **Graph structure:** $x \rightarrow y$ and $y \rightarrow x$
- **Strongly Connected Component:** {x, y} form one SCC since each can reach the other.


**The fundamental problem:** 
When we observe data P(x,y), we see their joint behavior under the feedback loop. But if we want to answer the causal question "What happens to y if I intervene and set x to a specific value?" - we need P(y | do(x)). 

**Key Question:** Can we identify P(y | do(x)) from observational data P(x,y) alone?

#### Example 2: More Complex Graph: Cycles and Confounders

![Cycle with Confounding](./graph%202%20with%20confounders.png)

**Complex cyclic structure with hidden causes:**

- **Directed cycle:** x → y → z → x (feedback loop)
- **Latent confounder:** x ↔ z (unmeasured common cause U)
- **Key challenges:** This graph has both feedback AND unmeasured confounding
- **Identifiable?** This depends on the structure; the generalized ID algorithm determines if this kind of graph is identifiable or not. (Success or Fail)

### 2. Working Examples: Using a Directed Mixed Graph (DMG) 

For this specific implementation, we will walkthrough the cyclic ID algorithm using a directed mixed graph (DMG) that exhibits some key challenges:

1. **Cycles** (feedback loops)
2. **Latent confounders** (unmeasured common causes)
3. **Self-loops**

#### 2.1 Graph Structure - Success Case 

![Example DMG with cycles, confounders, and self-loops](./DMG%20example%20.png)

This example graph contains:

